## bronzeテーブルのパイプラインをDatabricks Auto Loaderにて作成

In [0]:
%run ./00_config

In [0]:
src_file_path = f"/Volumes/test/temp_humidity_external/raw_data/temp-humidity_2025-04-09_07-53-39.csv"

schema = """
`Timestamp` STRING,
`Temperature` STRING,
`Humidity` STRING
"""


In [0]:
# CSV の中身をチェック
data = dbutils.fs.head(src_file_path)
print(data)

In [0]:
# Bronzeテーブルを作成
create_table_ddl = f"""
CREATE TABLE IF NOT EXISTS {bronze_table_path} (
    {schema},
    _rescued_data STRING,
    _datasource STRING,
    _ingest_timestamp timestamp
)
USING delta
"""
spark.sql(create_table_ddl)

In [0]:
# Databricks Auto Loader で利用するチェックポイントを初期化
dbutils.fs.rm(checkpoint_volume_path, True)

In [0]:
# csvデータをDeltaテーブルに取り込むようにAuto Loaderを構成
# `checkpoint_volume_path`変数を`cloudFiles.schemaLocation`に指定して、ソースからデータの読み込み処理を記述
df = (spark.readStream
      .format("cloudFiles")
      .option("cloudFiles.format", "csv")
      .option("cloudFiles.schemaLocation", checkpoint_volume_path)
      .option("cloudFiles.schemaHints", schema)
      .option("header", True)
      .load(src_file_path))

print(f"first_df:{df}")
print("------------------")

# 必要なカラムを選択
df = df.selectExpr("Timestamp", "`Temperature (℃)` as Temperature", "`Humidity (%)` as Humidity")

# メタデータ列を追加
df = df.select("*", "_metadata")

print(f"added_metadata_df:{df}")
print("------------------")

# メタデータ列に基づき監査列として、_datasource列と_ingest_timestamp列を追加
df = (
    df.select("*", "_metadata")
    .withColumn("_datasource", df["_metadata.file_path"])
    .withColumn("_ingest_timestamp", df["_metadata.file_modification_time"])
)

print(f"added_監査列_df:{df}")
print("------------------")

df = df.drop("_metadata")

In [0]:
# `checkpoint_volume_path`変数をチェックポイントとして指定して、書き込み
(
    df.writeStream.trigger(availableNow=True)
    .option("checkpointLocation", checkpoint_volume_path)
    .trigger(availableNow=True)
    .toTable(bronze_table_path)
)


In [0]:
# データが書き込まれたことを確認
display(spark.table(bronze_table_path))

## 事後処理

In [0]:
# ストリーム処理を停止
for stream in spark.streams.active:
    stream.stop()